# EXAMEN DATASCIENCE II

Antes de nada realizar un git clone del proyecto de github. Para la realización del test, borrar las opciones incorrectas dejando sólamente la respuesta correcta

# ARIKETA 1 (2 puntos)

Elige la respuesta mas adecuada para las siguientes preguntas. Nota: Respuesta correcta +0.20. Respuesta incorrecta -0.10

### NOTA: Llegado a este punto, tienes que realizar un git commit y un git push. Esta parte del notebook no se puede modificar y no se puede empezar con el ejercicio 2 sin haber hecho los pasos previos. Si se hace un cambio en el test a lo largo del examen, no se corregirán los ejercicios restantes.

# ARIKETA 2 (2 puntos)

Dada la página web https://dreguera.github.io y utilizando la Distancia de Levenshtein, printear en pantalla cuantas veces aparece el término escrito correctamente

In [3]:
from itertools import chain
import re
import string
import unidecode
import nltk
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize, sent_tokenize

from lxml import html
import requests
import codecs

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import Levenshtein
import gensim

import pandas as pd
import seaborn as sns
import numpy as np

In [4]:
#abrimos el html
with open(r'colores.html', "r") as f:
    page = f.read()

#creamos una matriz unidimensional de datos
tree = html.fromstring(page)

In [5]:
tree

<Element html at 0x218b7374810>

In [6]:
#cojo de la ariketa 1, dentro de la class = wrapper, la clase verde_claro, y cojo el texto
elemento = tree.xpath('//div[@id = "ariketa1"]/div[@class = "wrapper"]/div[@class="box verde_claro"]/text()')
elemento

['Java']

In [7]:
#cojo el texto con el id de muestra
#es el texto de donde tengo que coger las palabras que estan mal escritas
texto_importante = tree.xpath('//p[@id = "muestra"]/text()')
texto_importante

[' Pyhthon, Python, Pythhon, Matlan, Matlas, Matalab, Maltab, Jaava, Jara, Lava, Java, Spar, Sppark, Spaar, Sapakr, Scala, Slaca, Salac, Scalal, Clojutre, Clourem, Clojurr, Cutre, Gorvo, Grovvoy, Grrovy,Pyhthon, Python, Pythhon, Matlan, Matlas, Matalab, Maltab, Jaava, Jara, Lava, Java, Spar, Sppark, Spaar, Sapakr, Scala, Slaca, Salac, Scalal, Clojutre, Clourem, Clojurr, Cutre, Gorvo, Grovvoy, Grrovy,Pyhthon, Python, Pythhon, Matlan, Matlas, Matalab, Maltab, Jaava, Jara, Lava, Java, Spar, Sppark, Spaar, Sapakr, Scala, Slaca, Salac, Scalal, Clojutre, Clourem, Clojurr, Cutre, Gorvo, Grovvoy, Grrovy, Grovy']

In [8]:
for line in elemento:
    print(line)

Java


In [9]:
["java in python"]

['java in python']

In [10]:
for line in texto_importante:
    #print(line)
    for i in word_tokenize(line):
        
        print(i.lower())

pyhthon
,
python
,
pythhon
,
matlan
,
matlas
,
matalab
,
maltab
,
jaava
,
jara
,
lava
,
java
,
spar
,
sppark
,
spaar
,
sapakr
,
scala
,
slaca
,
salac
,
scalal
,
clojutre
,
clourem
,
clojurr
,
cutre
,
gorvo
,
grovvoy
,
grrovy
,
pyhthon
,
python
,
pythhon
,
matlan
,
matlas
,
matalab
,
maltab
,
jaava
,
jara
,
lava
,
java
,
spar
,
sppark
,
spaar
,
sapakr
,
scala
,
slaca
,
salac
,
scalal
,
clojutre
,
clourem
,
clojurr
,
cutre
,
gorvo
,
grovvoy
,
grrovy
,
pyhthon
,
python
,
pythhon
,
matlan
,
matlas
,
matalab
,
maltab
,
jaava
,
jara
,
lava
,
java
,
spar
,
sppark
,
spaar
,
sapakr
,
scala
,
slaca
,
salac
,
scalal
,
clojutre
,
clourem
,
clojurr
,
cutre
,
gorvo
,
grovvoy
,
grrovy
,
grovy


In [11]:
#creo una lista vacia
limpio_palabras = []
#de cada linea en 'elemento', pasa a minusculas
for line in elemento:
    limpio_palabras = ([i.lower() for i in word_tokenize(line)]) #word_tokenize te separa las palabras
    #me las mete en una lista
    
comparar = []
for line in texto_importante:
    comparar.append([w.lower() for w in word_tokenize(line)]) #con el append las meto a la lista

In [12]:
len(comparar[0])

157

In [76]:
limpio_palabras[0]

'java'

In [77]:
comparar[0][18]

'lava'

In [78]:
Levenshtein.distance(limpio_palabras[0], comparar[0][18])

1

In [79]:
#cantidad = []
veces = 0
#cuando la palabra es igual en la lista, es = 0 porque no hay que cambiar nada
#por eso le sumo un 1 cada vez que me de igual a 0, porque las palabras son iguales

#for i in range (de tal a tal), len es todo lo que recorre, la dimension de la lista, cuantos elementos tiene
for i in range(len(comparar[0])):
    if Levenshtein.distance(limpio_palabras[0], comparar[0][i]) == 0:
        veces +=  1
        
print('Basándonos en la distacia Levenshtein, la palabra Java aparece repetida', veces, 'veces')

Basándonos en la distacia Levenshtein, la palabra Java aparece repetida 3 veces


# ARIKETA 3 (4 puntos)

Vamos construir un sistema de recomendación de películas basandonos en la similitud del coseno. Para ello cargaremos el csv "movie_dataset.csv" un dataframe

In [18]:
df = pd.read_csv('movie_dataset.csv')
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


Una vez cargado el dataframe, vamos a realizar un simple preproceso, cambiando en las columnas ['keywords', 'cast', 'genres', 'director'] los valores NaN por ''.


In [19]:
df[['keywords', 'cast', 'genres', 'director']] = df[['keywords', 'cast', 'genres', 'director']].fillna('')

Entre todas estas características, las que nos interesan para encontrar la similitud para hacer la  recomendación son las siguientes: keywords, cast, genres y director. Para ello, vamos a crearnos una nueva columna df['combined_features'] concatenando los strings de las columnas keywords, cast, genres y director.

In [20]:
def combined_features(row):
    return row['keywords']+" "+row['cast']+" "+row['genres']+" "+row['director']
df["combined_features"] = df.apply(combined_features, axis =1)

In [21]:
df.combined_features

0       culture clash future space war space colony so...
1       ocean drug abuse exotic island east india trad...
2       spy based on novel secret agent sequel mi6 Dan...
3       dc comics crime fighter terrorist secret ident...
4       based on novel mars medallion space travel pri...
                              ...                        
4798    united states\u2013mexico barrier legs arms pa...
4799     Edward Burns Kerry Bish\u00e9 Marsha Dietlein...
4800    date love at first sight narration investigati...
4801     Daniel Henney Eliza Coupe Bill Paxton Alan Ru...
4802    obsession camcorder crush dream girl Drew Barr...
Name: combined_features, Length: 4803, dtype: object

La función anterior combinará todas nuestras features(palabras clave, elenco, géneros y director) de sus respectivas filas, y devolverá una fila con todas las características combinadas en un solo string.

Una vez creada esta nueva columna df["combined_features"], creamos la matriz de vectores y su matriz de similitud del coseno

In [22]:
df['combined_features']

0       culture clash future space war space colony so...
1       ocean drug abuse exotic island east india trad...
2       spy based on novel secret agent sequel mi6 Dan...
3       dc comics crime fighter terrorist secret ident...
4       based on novel mars medallion space travel pri...
                              ...                        
4798    united states\u2013mexico barrier legs arms pa...
4799     Edward Burns Kerry Bish\u00e9 Marsha Dietlein...
4800    date love at first sight narration investigati...
4801     Daniel Henney Eliza Coupe Bill Paxton Alan Ru...
4802    obsession camcorder crush dream girl Drew Barr...
Name: combined_features, Length: 4803, dtype: object

In [23]:
vectors = CountVectorizer().fit_transform(df['combined_features'].values)
vectors = vectors.toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [24]:
#va a ser una matriz cuadrada
#la primera fila va a ser el primer texto, la segunda fila el segundo texto....
#la primera columna va a ser el primer texto, la segunda columna va a ser el segundo texto...

csim = cosine_similarity(vectors)
csim

array([[1.        , 0.10540926, 0.12038585, ..., 0.        , 0.        ,
        0.        ],
       [0.10540926, 1.        , 0.0761387 , ..., 0.03651484, 0.        ,
        0.        ],
       [0.12038585, 0.0761387 , 1.        , ..., 0.        , 0.11145564,
        0.        ],
       ...,
       [0.        , 0.03651484, 0.        , ..., 1.        , 0.        ,
        0.04264014],
       [0.        , 0.        , 0.11145564, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.04264014, 0.        ,
        1.        ]])

In [25]:
len(csim[0])

4803

Si todo ha ido bien, la matriz de similitud debería tener esta pinta.

Ahora, debeis obtener del dataframe el index del título de la película escrapeado de la página https://dreguera.github.io. Acordaos de utilizar df.values para obtener el valor concreto.

In [26]:
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

In [27]:
get_index_from_title("Avatar") #me dice que es el indice 0 del df

0

In [28]:
#scrapear titulo
titulo = tree.xpath('//div[@id="ariketa2"]/div[@class="box verde_claro"]/text()')
t = titulo[0]
t

'Transformers'

In [29]:
get_index_from_title(t) #es lo mismo que abajo, cojo la funcion con el tirulo que he sacado dle scrapeo

111

In [30]:
movie_index = list(map(get_index_from_title, titulo))[0]
movie_index
#me dice cual es el indice del titulo

111

Esta función la debeis utilizar para obtener el índice del nombre de esta película. El índice lo deberíais guardar en una variable llamada por ejemplo movie_index

Los indices os deberían dar algo así:

In [31]:
print(t + ': ' + str(movie_index) ) #printea el resultado, que ya he sacado el titulo de la peli

Transformers: 111


Ahora, vamos a buscar las películas similares en la matriz de similitud. Para ello vamos a utilizar la siguiente función que me devuelve los indices de las películas y el coeficiente de similitud.



Para ello usaremos el movie_index de la película que hemos escrapeado. El método enumerate() agregará un contador a la lista iterable cosine_sim y la devolverá en forma de lista de películas similares con el puntaje de similitud de cada índice.

In [32]:
similar_movies = list(enumerate(csim[movie_index])) 
similar_movies

#me devuelve las que se parecen
#la 0 (pelicula de la fila 0) se parece a la mia en 0,16...
#Donde CSIM es vuestra matriz de similitudes y movie_index el indice que habeis calculado en el paso anterior

[(0, 0.16051447078102563),
 (1, 0.0761386987626881),
 (2, 0.1739130434782609),
 (3, 0.08178608201095308),
 (4, 0.2919201796799047),
 (5, 0.08340576562282992),
 (6, 0.0),
 (7, 0.22841609628806428),
 (8, 0.08891084489487741),
 (9, 0.17025130615174974),
 (10, 0.16681153124565984),
 (11, 0.08340576562282992),
 (12, 0.08695652173913045),
 (13, 0.08340576562282992),
 (14, 0.27583864218368526),
 (15, 0.12267912301642962),
 (16, 0.22470176588194152),
 (17, 0.08891084489487741),
 (18, 0.22841609628806428),
 (19, 0.08891084489487741),
 (20, 0.08512565307587487),
 (21, 0.08512565307587487),
 (22, 0.047836487323493986),
 (23, 0.04662524041201569),
 (24, 0.07372097807744857),
 (25, 0.0),
 (26, 0.15762208124782012),
 (27, 0.21281413268968719),
 (28, 0.17025130615174974),
 (29, 0.08891084489487741),
 (30, 0.08178608201095308),
 (31, 0.2085144140570748),
 (32, 0.11821656093586509),
 (33, 0.2364331218717302),
 (34, 0.0),
 (35, 0.6370220572706061),
 (36, 0.34050261230349943),
 (37, 0.04662524041201569),

Ahora, vamos a ordenar la lista creada similar_movies de manera descendente en base a su coeficiente de similitud del coseno. Para ello utilizar la siguiente función que lo único que hace es ordenar descendentemente de manera rápida y fácil

In [33]:
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)
sorted_similar_movies

#ordeno de las que mas se parecen a las que menos
#Debería mostraros algo tal que asi la lista sorted_similar_movies

[(111, 1.0000000000000002),
 (35, 0.6370220572706061),
 (52, 0.48900964692182586),
 (36, 0.34050261230349943),
 (4401, 0.3127716210856122),
 (242, 0.311187957132071),
 (3901, 0.3043478260869566),
 (4, 0.2919201796799047),
 (183, 0.2919201796799047),
 (41, 0.28625128703833574),
 (2444, 0.28625128703833574),
 (182, 0.28090032386679487),
 (972, 0.28090032386679487),
 (1001, 0.28090032386679487),
 (1326, 0.28090032386679487),
 (14, 0.27583864218368526),
 (1303, 0.27583864218368526),
 (46, 0.27104108243182823),
 (1192, 0.26215206019559845),
 (4077, 0.26086956521739135),
 (238, 0.2553769592276246),
 (270, 0.2553769592276246),
 (149, 0.25021729686848976),
 (260, 0.25021729686848976),
 (271, 0.25021729686848976),
 (400, 0.25021729686848976),
 (419, 0.25021729686848976),
 (2156, 0.25021729686848976),
 (83, 0.24573659359149527),
 (70, 0.24535824603285922),
 (661, 0.24535824603285922),
 (1482, 0.24535824603285922),
 (511, 0.24077170617153845),
 (3494, 0.24077170617153845),
 (33, 0.236433121871730

Ahora lo que debéis hacer es mostrar el título de las 15 primeras películas. Para ello, primero debeis obtener el título del df del indice y luego printearlo. 

In [34]:
get_title_from_index(index)

NameError: name 'get_title_from_index' is not defined

In [ ]:
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]

for movie in sorted_similar_movies[0:15]: #ACORDAOS QUE SOLO QUEREMOS LAS 15 PRIMERAS
    print(get_title_from_index(movie[0]))

# ARIKETA 4 (2 puntos)

Dados los ficheros noticia1.txt y noticia2.txt mediante la librería gensim devolver por pantalla (SÓLO) el porcentaje de similitud entre los dos textos. (Nota: Se pude hacer uso de las librerías vistas en clase para limpieza de texto)

In [44]:
import nltk
import gensim
import numpy as np
from nltk.tokenize import word_tokenize #separa las palabras
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\2002a\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\2002a\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [45]:
for i in stopwords:
    print (i)

TypeError: 'LazyCorpusLoader' object is not iterable

In [46]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

file_docs = []

with open ('noticia1.txt') as f:
    tokens = sent_tokenize(f.read())
    for line in tokens:
        file_docs.append(line)


In [47]:
file_docs

['Los vizcainos ya no podrÃ¡n moverse entre los territorios de la Comunidad AutÃ³noma Vasca y se cierran los municipios que se sitÃºan en zona roja, es decir, los que tienen una incidencia acumulada de casos positivos por coronavirus 19 en los Ãºltimos 14 dÃ\xadas que sea igual o superior a 500 por cada 100.000 habitantes.',
 'Por el momento en Bizkaia no se puede ni entrar ni salir de municipios como Bermeo, Lekeitio o Amorebieta ya que se sitÃºan en alerta roja.']

In [49]:
word_tokenize(line)

['Por',
 'el',
 'momento',
 'en',
 'Bizkaia',
 'no',
 'se',
 'puede',
 'ni',
 'entrar',
 'ni',
 'salir',
 'de',
 'municipios',
 'como',
 'Bermeo',
 ',',
 'Lekeitio',
 'o',
 'Amorebieta',
 'ya',
 'que',
 'se',
 'sitÃºan',
 'en',
 'alerta',
 'roja',
 '.']

In [67]:
sentences = file_docs

words_per_sentence_clean = []
for line in sentences:
    query_doc = [w.lower() for w in word_tokenize(line)]
    words_per_sentence_clean.append([w for w in query_doc if w not in string.punctuation]) #borra signos de puntuacion

In [69]:
words_per_sentence_clean

[['los',
  'vizcainos',
  'ya',
  'no',
  'podrã¡n',
  'moverse',
  'entre',
  'los',
  'territorios',
  'de',
  'la',
  'comunidad',
  'autã³noma',
  'vasca',
  'y',
  'se',
  'cierran',
  'los',
  'municipios',
  'que',
  'se',
  'sitãºan',
  'en',
  'zona',
  'roja',
  'es',
  'decir',
  'los',
  'que',
  'tienen',
  'una',
  'incidencia',
  'acumulada',
  'de',
  'casos',
  'positivos',
  'por',
  'coronavirus',
  '19',
  'en',
  'los',
  'ãºltimos',
  '14',
  'dã\xadas',
  'que',
  'sea',
  'igual',
  'o',
  'superior',
  'a',
  '500',
  'por',
  'cada',
  '100.000',
  'habitantes'],
 ['por',
  'el',
  'momento',
  'en',
  'bizkaia',
  'no',
  'se',
  'puede',
  'ni',
  'entrar',
  'ni',
  'salir',
  'de',
  'municipios',
  'como',
  'bermeo',
  'lekeitio',
  'o',
  'amorebieta',
  'ya',
  'que',
  'se',
  'sitãºan',
  'en',
  'alerta',
  'roja']]

In [70]:
for line in sentences:
    for w in word_tokenize(line):
        print(w)
        
        words_per_sentence_clean.append([w for w in query_doc if w not in string.punctuation])

Los
vizcainos
ya
no
podrÃ¡n
moverse
entre
los
territorios
de
la
Comunidad
AutÃ³noma
Vasca
y
se
cierran
los
municipios
que
se
sitÃºan
en
zona
roja
,
es
decir
,
los
que
tienen
una
incidencia
acumulada
de
casos
positivos
por
coronavirus
19
en
los
Ãºltimos
14
dÃ­as
que
sea
igual
o
superior
a
500
por
cada
100.000
habitantes
.
Por
el
momento
en
Bizkaia
no
se
puede
ni
entrar
ni
salir
de
municipios
como
Bermeo
,
Lekeitio
o
Amorebieta
ya
que
se
sitÃºan
en
alerta
roja
.


In [51]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [52]:
sentences

['Los vizcainos ya no podrÃ¡n moverse entre los territorios de la Comunidad AutÃ³noma Vasca y se cierran los municipios que se sitÃºan en zona roja, es decir, los que tienen una incidencia acumulada de casos positivos por coronavirus 19 en los Ãºltimos 14 dÃ\xadas que sea igual o superior a 500 por cada 100.000 habitantes.',
 'Por el momento en Bizkaia no se puede ni entrar ni salir de municipios como Bermeo, Lekeitio o Amorebieta ya que se sitÃºan en alerta roja.']

In [53]:
words_per_sentence_clean

[['los',
  'vizcainos',
  'ya',
  'no',
  'podrã¡n',
  'moverse',
  'entre',
  'los',
  'territorios',
  'de',
  'la',
  'comunidad',
  'autã³noma',
  'vasca',
  'y',
  'se',
  'cierran',
  'los',
  'municipios',
  'que',
  'se',
  'sitãºan',
  'en',
  'zona',
  'roja',
  'es',
  'decir',
  'los',
  'que',
  'tienen',
  'una',
  'incidencia',
  'acumulada',
  'de',
  'casos',
  'positivos',
  'por',
  'coronavirus',
  '19',
  'en',
  'los',
  'ãºltimos',
  '14',
  'dã\xadas',
  'que',
  'sea',
  'igual',
  'o',
  'superior',
  'a',
  '500',
  'por',
  'cada',
  '100.000',
  'habitantes'],
 ['por',
  'el',
  'momento',
  'en',
  'bizkaia',
  'no',
  'se',
  'puede',
  'ni',
  'entrar',
  'ni',
  'salir',
  'de',
  'municipios',
  'como',
  'bermeo',
  'lekeitio',
  'o',
  'amorebieta',
  'ya',
  'que',
  'se',
  'sitãºan',
  'en',
  'alerta',
  'roja']]

In [54]:
#da un numero a cada palabra
dicti = gensim.corpora.Dictionary(words_per_sentence_clean)
print(dicti.token2id)

{'100.000': 0, '14': 1, '19': 2, '500': 3, 'a': 4, 'acumulada': 5, 'autã³noma': 6, 'cada': 7, 'casos': 8, 'cierran': 9, 'comunidad': 10, 'coronavirus': 11, 'de': 12, 'decir': 13, 'dã\xadas': 14, 'en': 15, 'entre': 16, 'es': 17, 'habitantes': 18, 'igual': 19, 'incidencia': 20, 'la': 21, 'los': 22, 'moverse': 23, 'municipios': 24, 'no': 25, 'o': 26, 'podrã¡n': 27, 'por': 28, 'positivos': 29, 'que': 30, 'roja': 31, 'se': 32, 'sea': 33, 'sitãºan': 34, 'superior': 35, 'territorios': 36, 'tienen': 37, 'una': 38, 'vasca': 39, 'vizcainos': 40, 'y': 41, 'ya': 42, 'zona': 43, 'ãºltimos': 44, 'alerta': 45, 'amorebieta': 46, 'bermeo': 47, 'bizkaia': 48, 'como': 49, 'el': 50, 'entrar': 51, 'lekeitio': 52, 'momento': 53, 'ni': 54, 'puede': 55, 'salir': 56}


In [55]:
#veces que sale cada palabra
corpus = [dicti.doc2bow(gen_doc) for gen_doc in words_per_sentence_clean] 
corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 2),
  (13, 1),
  (14, 1),
  (15, 2),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 5),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 2),
  (29, 1),
  (30, 3),
  (31, 1),
  (32, 2),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1)],
 [(12, 1),
  (15, 2),
  (24, 1),
  (25, 1),
  (26, 1),
  (28, 1),
  (30, 1),
  (31, 1),
  (32, 2),
  (34, 1),
  (42, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 2),
  (55, 1),
  (56, 1)]]

In [56]:
tf_idf = gensim.models.TfidfModel(corpus)
for doc in tf_idf[corpus]:
    print([[dicti[id], np.around(freq, decimals=2)] for id, freq in doc])

[['100.000', 0.13], ['14', 0.13], ['19', 0.13], ['500', 0.13], ['a', 0.13], ['acumulada', 0.13], ['autã³noma', 0.13], ['cada', 0.13], ['casos', 0.13], ['cierran', 0.13], ['comunidad', 0.13], ['coronavirus', 0.13], ['decir', 0.13], ['dã\xadas', 0.13], ['entre', 0.13], ['es', 0.13], ['habitantes', 0.13], ['igual', 0.13], ['incidencia', 0.13], ['la', 0.13], ['los', 0.66], ['moverse', 0.13], ['podrã¡n', 0.13], ['positivos', 0.13], ['sea', 0.13], ['superior', 0.13], ['territorios', 0.13], ['tienen', 0.13], ['una', 0.13], ['vasca', 0.13], ['vizcainos', 0.13], ['y', 0.13], ['zona', 0.13], ['ãºltimos', 0.13]]
[['alerta', 0.26], ['amorebieta', 0.26], ['bermeo', 0.26], ['bizkaia', 0.26], ['como', 0.26], ['el', 0.26], ['entrar', 0.26], ['lekeitio', 0.26], ['momento', 0.26], ['ni', 0.52], ['puede', 0.26], ['salir', 0.26]]


In [62]:
sims = gensim.similarities.Similarity('b/',tf_idf[corpus],
                                        num_features=len(dicti))

In [63]:
file2_docs = []

with open ('noticia2.txt', encoding="utf8") as f:
    tokens = sent_tokenize(f.read())
    for line in tokens:
        file2_docs.append(line)

print("Number of documents:",len(file2_docs))  
for line in file2_docs:
    query_doc = [w.lower() for w in word_tokenize(line)]
    query_doc = [w for w in query_doc if w not in string.punctuation]
    query_doc_bow = dicti.doc2bow(query_doc) 

Number of documents: 2


In [64]:
query_doc_tf_idf = tf_idf[query_doc_bow]

In [65]:
import numpy as np

sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
print(sum_of_sims)

0.6882553


In [66]:
print('Porcentaje de similitud entre noticia1 y noticia 2 es de %s' % float(sum_of_sims/len(sentences) * 100) + '%')

Porcentaje de similitud entre noticia1 y noticia 2 es de 34.41276550292969%
